In [1]:
import numpy as np
import pandas as pd
import re
import glob
import os
import sys

In [2]:
ref = "SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
avgfp = pd.read_csv("avgfp.tsv", sep = "\t")

In [8]:

for mut in avgfp.variant[:20000]:
    with open("./20K_input_fasta/"+mut+".fasta", "w") as f:
        muts = mut.split(",")
        #f.write(">"+mut+"\n")
        if len(muts)>1:

            variant = ref
            for m in muts:

                pos = int(re.findall(r'\d+', m)[0])


                variant= variant[:pos] + m[-1]+variant[pos+1:]
            f.write(variant+"\n")
        else:
            pos = int(re.findall(r'\d+', muts[0])[0])

            variant = ref
            variant= variant[:pos] + muts[0][-1]+variant[pos+1:]
            f.write(variant+"\n") 


In [6]:
avgfp.variant[:10000]

0                               A108D
1       A108D,N144D,I186V,M231T,L234P
2             A108D,Y141N,H146L,E220G
3                               A108G
4             A108G,K138M,R166C,L234P
                    ...              
9995                      F128I,T228S
9996                      F128I,V148I
9997                            F128L
9998                      F128L,A152V
9999          F128L,D131V,Y198C,A224T
Name: variant, Length: 10000, dtype: object

## Extract pssm info

In [2]:
test = np.loadtxt("10K_out_pssm/A108D.fasta.pssm",
                 skiprows=3,usecols=np.arange(2,22), max_rows=238)
test_str = np.genfromtxt("10K_out_pssm/A108D.fasta.pssm",
                      skip_header=3, usecols =[1],max_rows = 237, dtype='str')
aas = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L",
         "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]

In [90]:
def transform_pssm_400(filepath):
    pssm = np.loadtxt(filepath, skiprows=3,usecols=np.arange(2,22), max_rows=238)
    seq = np.genfromtxt(filepath, skip_header=3, usecols =[1],max_rows = 237, dtype='str')
    pssm_transformed = np.zeros((20,20))
    for i in range(20):
        residues = np.where(test_str==aas[i])[0]
        pssm_transformed[i] = pssm[residues,:].sum(axis = 0)
    pssm_transformed = pssm_transformed/len(seq)
    max_sc = np.max(pssm_transformed)
    min_sc = np.min(pssm_transformed)
    pssm_transformed = (pssm_transformed-min_sc)/(max_sc-min_sc)
    return pssm_transformed
        

In [93]:
transform_pssm_400("10K_out_pssm/A108D,Y141N,H146L,E220G.fasta.pssm").shape

(20, 20)

In [108]:
all_fp = glob.glob("./10K_out_pssm/*")
for i in range(len(f[:3])):
    fname = os.path.basename(all_fp[i]).split(".")[0]
    trans_pssm = transform_pssm_400(f[i])
    np.save("./10K_feat_extract/"+fname+".npy",trans_pssm)
        

In [107]:
os.path.basename(all_fp[i]).split(".")

['D115G,S145G,I186N,S203Y,L205P', 'fasta', 'pssm']

In [101]:
os.path.basename(f[1])

'D74G,I121F.fasta.pssm'

In [102]:
len(f)

8387